In [1]:
import numpy as np
import pandas as pd
import os
import random

In [2]:
input_path = '../datasets/'

log_df = pd.read_csv('../output/cart_log.csv')
child_item_df = pd.read_csv('../output/child_item.csv')
product_master_df = pd.read_csv(os.path.join(input_path, 'product_master.csv'))
meta_df = pd.read_csv(os.path.join(input_path, 'meta.csv'))

/Users/imazekishouta/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
is_predict= True # True

In [4]:
def get_paid_event(df):
    print('all sessions: ', len(df.session_id.unique()))

    paid_event_ids = list(df[df['is_payment']==1]['session_id'].unique())
    print('num sessions paid: ', len(paid_event_ids))

    return df[df['session_id'].isin(paid_event_ids)]

paid_event_df = get_paid_event(log_df)

all sessions:  720175
num sessions paid:  618462


In [5]:
def get_item_record(df, cols=['session_id', 'value_1', 'n_items', 'predict']):
    return df[df['kind_1']=='商品'][cols]

paid_item_event_df = get_item_record(paid_event_df)
paid_item_event_df.head()

,session_id,value_1,n_items,predict
1,0,4522646719765,1,0
2,0,4522646695663,1,0
3,0,4522646716641,1,0
4,0,4522646695670,1,0
5,0,4522646720440,1,0


In [6]:
def join_category_id(df, child_item_df):
    df['value_1'] = df['value_1'].astype('int64')
    df = pd.merge(df, child_item_df[['JAN', 'category_id', '商品名']], left_on='value_1', right_on='JAN', how='inner') # 紐づかないものは全て落とす
    return df.drop(['value_1', 'JAN'], axis=1)

category_event_df = join_category_id(paid_item_event_df, child_item_df)
category_event_df.head()

,session_id,n_items,predict,category_id,商品名
0,8,1,0,136,アポロマーブルアソート袋
1,50,1,0,136,アポロマーブルアソート袋
2,64,1,0,136,アポロマーブルアソート袋
3,76,1,0,136,アポロマーブルアソート袋
4,242,1,1,136,アポロマーブルアソート袋


In [7]:
def get_target_category_record(df, is_predict=True):
    if is_predict:
        df = df[df['predict']==0] # 見えてるところの情報だけにする
    return df.drop(['predict'], axis=1)

target_row_event_df = get_target_category_record(category_event_df, is_predict=is_predict)
target_row_event_df.head()

,session_id,n_items,category_id,商品名
0,8,1,136,アポロマーブルアソート袋
1,50,1,136,アポロマーブルアソート袋
2,64,1,136,アポロマーブルアソート袋
3,76,1,136,アポロマーブルアソート袋
5,791,1,136,アポロマーブルアソート袋


In [8]:
def calc_buy(df):
    df = df.groupby(['session_id'])[['n_items']].sum().reset_index()
    df.columns = ['session_id', 'child_items']
    return df
target_item_sum_df = calc_buy(target_row_event_df)
target_item_sum_df.head()

,session_id,child_items
0,8,1
1,21,2
2,50,1
3,64,4
4,72,3


In [9]:
def join_meta_df(df, meta_df):
    session_df = meta_df[['session_id']]
    df = pd.merge(session_df, df, on='session_id', how='left')
    df.fillna(0, inplace=True)
    return df

all_sessions_Y = join_meta_df(target_item_sum_df, meta_df)
all_sessions_Y.head()

,session_id,child_items
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0


In [10]:
if is_predict:
    all_sessions_Y.to_csv('../output/given_train_child_feature.csv', index=None)
else:
    all_sessions_Y.to_csv('../output/all_train_child_feature.csv', index=None)
